## Threads

#### Thread class anotation by python

In [ ]:
from threading import Thread

print(help(Thread))

#### Thread usage example

In [ ]:
from threading import Thread
from time import sleep

WORK = True


def handler(name, sleep_time):
    while WORK:
        print(f'{name} start to sleep {sleep_time} sec')
        sleep(sleep_time)

In [ ]:
thread1 = Thread(target=handler, args=('first', 2))
thread2 = Thread(target=handler, args=('second', 3))

In [ ]:
thread1.start()
thread2.start()

print('Start to sleep')
sleep(10)

print('Stop work')
WORK = False

#### Join method usage example

In [ ]:
from threading import Thread
from time import sleep

def block():
    print('Start to block')
    sleep(2)
    print('Unblock')

In [ ]:
thread1 = Thread(target=block)
thread1.start()
print("I'm here 1")
sleep(5)
print("Now I'm here")
thread1.join()

In [ ]:
thread1 = Thread(target=block)
thread2 = Thread(target=block)
thread1.start()
thread1.join()
thread2.start()
thread2.join()  # wait thread
print("I'm here 1")
sleep(5)
print("Now I'm here")

#### Custom thread class

In [ ]:
from threading import Thread

class MyThread(Thread):
    def run(self):
        print(f'Run thread: {self.name}')

MyThread(name='Thread1').start()
MyThread(name='Thread2').start()

#### Example of concurrent number calculation using threads

In [ ]:
from threading import Thread

NUM = 0

def add():
    global NUM
    for i in range(1000000):
        NUM += 1

t1 = Thread(target=add)
t2 = Thread(target=add)
t1.start()
t2.start()
t1.join()
t2.join()

print(NUM) # result?

In [ ]:
import dis
a = 5
b = 7

dis.dis('a += b')

## Synchronization Primitives

#### Lock usage example

In [ ]:
from threading import Lock, Thread
from time import sleep


class Bank:
    money = 0
    count = 1
    lock = Lock()

    def replenish(self, s=10):
        self.lock.acquire()
        print(Bank.money, 'money')
        sleep(5)
        Bank.money += s
        print('Replenish cash')
        self.lock.release()

    def withdraw(self, s=10):
        self.lock.acquire()
        self.count += 1
        print(Bank.money, 'money')
        sleep(1)
        if self.money >= s:
            Bank.money -= s
            print('Withdraw cash')
        else:
            print('Don\'t enough money')
        self.lock.release()


class User(Thread):
    def __init__(self, name, func):
        super().__init__(name=name)
        self.name = name
        self.func = func

    def run(self):
        print(f'{self.name} try to {self.func}')
        bank = Bank()
        func = getattr(bank, self.func)
        func()
        print(f'{self.name} made {self.func} operation')

In [ ]:
Bank.money = 20
Bank.count = 0

user1 = User('Bob', 'withdraw')
user1.start()
sleep(1)
user2 = User('Kate', 'withdraw')
user2.start()

user1.join()
user2.join()

print(Bank.money, 'money')
print(Bank.count, 'count')

#### Example of deadlock

In [ ]:
from threading import Lock, Thread
from time import sleep


class Bank:
    def __init__(self, name, money=100):
        self.name = name
        self.money = money
        self.lock = Lock()

    def replenish(self, s=10):
        print(self.money, 'money:', self.name)
        self.money += s
        print('Replenish cash:', self.name)

    def withdraw(self, s=10):
        print(self.money, 'money:', self.name)
        sleep(1)
        self.money -= s
        print('Withdraw cash:', self.name)
            
    def transfer(self, friend_bank, s=10):
        with self.lock:
            print('Lock myself: ', self.name)
            self.withdraw(s)
        with friend_bank.lock:
            print('Lock friend: ', friend_bank.name)
            friend_bank.replenish(s)

class User(Thread):
    def __init__(self, name, bank, friend_bank):
        super().__init__(name=name)
        self.name = name
        self.bank = bank
        self.friend_bank = friend_bank

    def run(self):
        print(f'{self.name} try to transfer to {self.friend_bank.name}')
        self.bank.transfer(self.friend_bank)
        print(f'{self.name} made transfer operation to {self.friend_bank.name}')

In [ ]:
bank1 = Bank('Bob')
bank2 = Bank('Kate')
bank3 = Bank('Ann')

user1 = User('Bob', bank1, bank2)
user2 = User('Kate', bank2, bank3)
user3 = User('Ann', bank3, bank2)

user1.start()
user2.start()
user3.start()

user1.join()
user2.join()
user3.join()

print(bank1.money)
print(bank2.money)
print(bank3.money)

## GIL

In [ ]:
def multiply():
    res = 1
    for i in range(1, 100000):
        res *= i

In [ ]:
from time import time

t1 = time()
multiply()
print(time() - t1)

In [ ]:
from threading import Thread
from time import time

th1 = Thread(target=multiply)
th2 = Thread(target=multiply)

t1 = time()
th1.start()
th2.start()
th1.join()
th2.join()
print(time() - t1)

In [ ]:
# For what
import requests

websites = ['https://www.python.org/'] * 100
result = []

def handle(website):
    global result
    response = requests.get(website)
    result.append(response.status_code)

In [ ]:
from time import time

result.clear()

t1 = time()
for website in websites:
    handle(website)
print(time() - t1)
print(len(result))

In [ ]:
from threading import Thread
from time import time

result.clear()

threads = [Thread(target=handle, args=(website,)) for website in websites]
t1 = time()
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
print(time() - t1)
print(len(result))

## Processes

#### Process class python annotation

In [ ]:
from multiprocessing import Process

print(help(Process))

#### Process usage example

In [ ]:
from multiprocessing import Process
from time import sleep

p1 = Process(target=sleep, args=(600,), name='MyTestProcess1')
p2 = Process(target=sleep, args=(600,), name='MyTestProcess2')

p1.start()
p2.start()

p1.kill()
p2.terminate()

#### Custom process class

In [ ]:
from multiprocessing import Process
from time import sleep

class MyProcess(Process):
    def run(self):
        sleep(1)
        print('Done')

MyProcess().start()
print('I am here')

#### Pool usage

In [ ]:
import requests

websites = ['https://www.python.org/'] * 100

def handle(website):
    response = requests.get(website)
    
    return response.status_code

In [ ]:
from multiprocessing import Pool
from time import time

count = 10000000
t1 = time()
with Pool(count) as pool:
    result = pool.map(handle, websites)
print(time() - t1)
    
print(len(result))

#### Example that shows that processes are isolated

In [ ]:
from multiprocessing import Process

NUM = 0

def add():
    global NUM
    for i in range(1000000):
        NUM += 1
    
    print(NUM)

p1 = Process(target=add)
p2 = Process(target=add)
p1.start()
p2.start()
p1.join()
p2.join()

print(NUM)

In [ ]:
from multiprocessing import Process, Value
from ctypes import c_int

NUM = Value(c_int, 0)

def add():
    global NUM
    for i in range(1000000):
        NUM.value += 1

p1 = Process(target=add)
p2 = Process(target=add)
p1.start()
p2.start()
p1.join()
p2.join()

print(NUM.value)